In [663]:
head = 'l30_r15'
directory = '../data/'
model_fname ='../models/Logistic + L2 (C=1.00) norm l30_r15.pkl'


import pickle    
from py.utils import load_data

X, y, x_words, vocabs = load_data(head, directory)
with open(model_fname, 'rb') as f:
    classifier = pickle.load(f)

x shape = (15715, 4551)
y shape = (15715,)
# features = 4551
# L words = 15715


In [2]:
coefficient = {vocabs[j]:coef for j, coef in enumerate(classifier.coef_[0])}

In [8]:
list(coefficient.items())[:5]

[('어오고', -0.0035026965225787533),
 ('어놓는', -0.0017447214860182797),
 ('느냐고', -0.027019952035748179),
 ('라느니', 0.0013030948275454116),
 ('으셔서', -0.3228328145095205)]

In [580]:
import sys
from soynlp.utils import get_process_memory
from soynlp.hangle import normalize
import numpy as np

In [583]:
# with open(sentence_fname, encoding='utf-8') as f:
#     with open('../data/sentences_onlyhangle.txt', 'w', encoding='utf-8') as fo:        
#         for sent in f:
#             fo.write('%s\n' % normalize(sent.strip()))
# print('done')

done


In [276]:
class TrainedNounExtractor:
    def __init__(self, coefficient, max_length=8):
        self._coef = coefficient
        self.lmax = max_length
        
    def extract(self, sents, min_count=10, min_noun_score=0.1):
        self.lrgraph, self.lset, self.rset = self._build_lrgraph(sents, min_count)
        self.lentropy, self.rentropy = self._branching_entropy(lrgraph)
        scores = self._compute_noun_score(self.lrgraph)
        return scores, self.lrgraph
        scores = self._postprocessing(lrgraph, scores)

    def _build_lrgraph(self, sents, min_count, pruning_min_count=2):
        lset = {}
        rset = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    lset[l] = lset.get(l, 0) + 1
                    rset[r] = rset.get(r, 0) + 1
            if n_sent % 1000 == 999:
                args = (n_sent+1, len(lset), len(rset), get_process_memory())
                sys.stdout.write('\rscaning vocabulary ... %d sents #(l= %d, r= %d), mem= %.3f Gb' % args)
            if n_sent % 500000 == 499999:
                lset = {l:f for l,f in lset.items() if f >= pruning_min_count}
                rset = {l:f for l,f in rset.items() if f >= pruning_min_count}
        lset = {l:f for l,f in lset.items() if f >= min_count}
        rset = {l:f for l,f in rset.items() if f >= min_count}
        
        n_sents = n_sent
        
        lrgraph = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    if not (l in lset) or not (r in rset):
                        continue
                    rdict = lrgraph.get(l, {})
                    rdict[r] = rdict.get(r, 0) + 1
                    lrgraph[l] = rdict            
            if n_sent % 1000 == 999:
                args = (100*(n_sent+1)/n_sents, '%', n_sent+1, n_sents, get_process_memory())
                sys.stdout.write('\rbuilding lrgraph ... (%.3f %s, %d in %d), mem= %.3f Gb' % args)
        args = (len(lset), len(rset), sum((len(rdict) for rdict in lrgraph.values())), get_process_memory())
        print('\rlrgraph has been built. (#L= %d, #R= %d, #E=%d), mem= %.3f Gb' % args)
        return lrgraph, lset, rset
    
    def _branching_entropy(self, lrgraph):
        from collections import defaultdict
        def entropy(d):
            sum_ = sum(d.values())
            if sum_ == 0: return 0
            return -1 * sum((v/sum_) * np.log(v/sum_) for v in d.values())
        def branch_map(d, get_branch):
            b = defaultdict(lambda: 0)
            for ext,f in d.items():
                if ext != '':
                    b[get_branch(ext)] += f
            return b
        def all_entropy(graph, get_branch=lambda x:x[0]):
            return {w:entropy(branch_map(d, get_branch)) for w, d in graph.items()}
        def to_rlgraph(lrgraph):
            rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
            for l, rdict in lrgraph.items():
                for r, f in rdict.items():
                    if r == '': continue
                    rlgraph[r][l] += f
            return {r:dict(ldict) for r, ldict in rlgraph.items()}    
        print('compute branching entropy ...', end='')
        lentropy = all_entropy(lrgraph)
        rentropy = all_entropy(to_rlgraph(lrgraph), get_branch=lambda x:x[-1])
        print(' done')
        return lentropy, rentropy
        
    def _compute_noun_score(self, lrgraph):
        from collections import namedtuple
        Score = namedtuple('Score', 'score frequency branching_entropy feature_fraction eojeol_fraction')
        scores = {}
        n = len(lrgraph)
        for i, (l, rdict) in enumerate(lrgraph.items()):
            rdict_ = {r:f for r,f in rdict.items() if r in self._coef}
            rsum = sum((f for r,f in rdict.items() if r != ''))
            frequency = self.lset.get(l, 0)
            feature_fraction = sum(rdict_.values()) / rsum if rsum > 0 else 0
            eojeol_fraction = rdict.get('', 0) / frequency
            if not rdict_:
                score = 0
            else:
                score = sum(f*self._coef[r] for r, f in rdict_.items()) / sum(rdict_.values())
            scores[l] = Score(score, frequency, self.lentropy.get(l, 0), feature_fraction, eojeol_fraction)
            if (i+1) % 1000 == 0:
                args = (100*(i+1)/n, '%', i+1, n)
                sys.stdout.write('\rcompute noun score ... (%.3f %s, %d in %d)' % args)
        print('\rcomputing noun score has been done.')
        return scores
#         return sorted(scores.items(), key=lambda x:x[1].score, reverse=True)
        
    def _postprocessing(self, lrgraph, scores):
        print('hi')

In [584]:
from config import sentence_fname
class Sentences:
    def __init__(self, fname):
        self.fname = fname
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip()

sentences = Sentences('../data/sentences_onlyhangle.txt')
noun_extractor = TrainedNounExtractor(coefficient)
scores, lrgraph = noun_extractor.extract(sentences)

lset, rset = noun_extractor.lset, noun_extractor.rset

lrgraph has been built. (#L= 173620, #R= 71297, #E=2022472), mem= 2.100 Gb
compute branching entropy ... done
computing noun score has been done.


In [585]:
def to_rlgraph(lrgraph):
    from collections import defaultdict
    rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
    for l, rdict in lrgraph.items():
        for r, f in rdict.items():
            if r == '': continue
            rlgraph[r][l] += f
    return {r:dict(ldict) for r, ldict in rlgraph.items()}

rlgraph = to_rlgraph(lrgraph)

## Preprocessor, Postprocessing의 필요성

- lrgraph를 만들 때 한글 외의 기호를 삭제한 어절로 이뤄져야 함 
- N = Nsub + J problem
    - 중세보편주 + {의, 의의}
- [N+Jsub] + J problem
    - 대학생과 + {'', 의}
    - {의, 과의} 모두 조사
- 1음절 명사들은 제외하였다. 필요하면 그 때 추가하도록. 아래는 0.5 를 넘은 1음절 명사들이다 

    질 럼 듀 균 흑 첨 응 쌘 초 삥 휭 릴 변 문 핸 윈 심 팅 뉴 를 맏 갱 흥 국 결 잣 휴 콤 도 닉 퓌 쫌 넷 놋 깡 앨 캡 불 횡 완 샤 원 삽 당 비 총 서 시 튜 골 ㅅ 판 범 옥 꽤 껀 빈 란 장 셰 징 산 송 쎈 덴 쑥 쉼 냇 펑 택 밸 군 외 표 멍 볼 카 렌 평 딱 쟁 헛 웹 멧 귄 띵 탈 망 번 케 절 퍽 약 갯 깃 그 셋 피 캠 뿐 턴 롤 흔 겹 빅 철 간 촌 령 측 꽝 겸 논 암 뚝 욱 얀 뭣 취 껄 칭 때 면 쇄 쇳 안 잠 푼 종 말 쌈 차 성 침 길 의 칠 호 향 전 형 킴 흉 패 둘 엠 끝 양 넙 람 옴 악 통 싯 팝 수 좀 새 배 눔 쿵 채 회 똔 괄 림 윷 벱 발 벨 랑 날 일 분 쯔 찡 읍 콘 픽 편 헉 극 셈 햄 관 제 팽 또 팬 권 쾅 땐 흠 너 격 탕 뻑 훤 목 휠 병 휑 즤 후 김 톱 잔 숀 쿨 욕 탭 묘 젤 방 뻔 잼 것 누 맛 급 닌 갓 겉 척 끗 링 화 탓 멜 못 잭 내 곡 답 강 족 걔 법 뻥 킬 귤 년 뽁 중 애 득 책 셸 루 빤 죄 곰 똘 멱 꾀 뜻 궤 쟤 늬 록 놈 쿡 룰 폼 폭 옻 션 펠 플 윌 캔 브 갠 뜸 칩 홑 덕 앞 월 뚱 앳 섬 왕 힘 숄 론 글 돛 밥 맘 똥 키 대 식 위 맥 틱 점 떡 룸 빽 값 꼿 빛 폴 늠 색 쇠 움 봄 집 뼘 바 쯤 딸 씽 꽃 뭔 곳 첩 손 락 딕 껌 빵 톰 뿔 잽 껩 술 짬 획 숍 녘 냥 펙 앰 꿈 탑 숯 벽 씩 량 층 갭 잎 펌 겐 북 착 딘 팩 힐 펜 리 벼 곽 삯 프 티 핵 댁 홈 옷 칡 별 꿩 뇌 춤 예 턱 밤 뽕 쪽 짐 개 씨 삔 굿 꼴 콩 붐 팥 솜 샬 넋 칸 밭 쉿 휙 꾹 돈 칼 팀 니 듯 빔 샘 눈 랩 몫 펄 컵 둑 툴 틈 꿀 뱀 컷 쉘 씬 흙 팁 큐 훅 앎 늪 숲 속 흐 뫼 샅 돔 뒤 뼈 쌀 닭 코 곁 뺨 쨍 볏 샷 뽈 뮤 쟝 밑 괌 홀 괘 롬 짝 맴 명 빰 녜 싹 팸 퀭 험 땀 닢 덤 볕 슛 뜰 몸 땅 즙 킥 홉 닻 뭍 립 밖 솥 쑈 퀸 텔 덫 옆 쇼 퉤 쌍 댐 룩 으 램 땜 닐 귀 콕 톤 첼 등 숱 텀 샨 쿤 섶 력 꾐 윙 씰 훠 쩜 엥 콰 캉 팰 삶 님 킹 네 므 류 괜 톨 혀 홰 웜 녀 갸

In [586]:
# useful function
def pretty(namedtuple_instance, end='\n'):
    print('%s(%s)' % (namedtuple_instance.__class__.__name__, ', '.join(['%s=%d' % (field, value) if (type(value) == int or type(value) == np.int) else '%s=%.3f' % (field, value)  for field, value in namedtuple_instance._asdict().items()])),end=end )

def pretty_list(l):
    print('[', end='')
    for w, nt in l[:10]:
        print('(%s,'%w, end=' ')
        pretty(nt, end='),\n')
    print(']' if len(l) <= 10 else '... ]')

In [587]:
lrgraph['중세보편주']

{'의': 6, '의를': 7, '의의': 2}

In [588]:
lrgraph['대학생과']

{'': 25, '의': 1}

In [589]:
sorted(lrgraph['대학생'].items(), key=lambda x:x[1], reverse=True)[:10]

[('', 206),
 ('이', 76),
 ('들이', 76),
 ('들의', 68),
 ('들은', 39),
 ('을', 29),
 ('의', 26),
 ('과', 25),
 ('들', 24),
 ('들을', 23)]

In [629]:
pretty_list(list(filter(lambda x:x[1].frequency > 100 and len(x[0]) > 1, scores))[:3])

[(고전주, Score(score=6.211, frequency=134, branching_entropy=-0.000, feature_fraction=0.364, eojeol_fraction=0.000)),
(개인과, Score(score=6.211, frequency=128, branching_entropy=0.562, feature_fraction=0.750, eojeol_fraction=0.969)),
(교수와, Score(score=6.211, frequency=118, branching_entropy=0.451, feature_fraction=1.000, eojeol_fraction=0.958)),
]


In [630]:
pretty_list(list(scores)[:3])

[(고전주, Score(score=6.211, frequency=134, branching_entropy=-0.000, feature_fraction=0.364, eojeol_fraction=0.000)),
(교사로서, Score(score=6.211, frequency=30, branching_entropy=0.377, feature_fraction=1.000, eojeol_fraction=0.467)),
(원리주, Score(score=6.211, frequency=10, branching_entropy=-0.000, feature_fraction=0.111, eojeol_fraction=0.000)),
]


## Dev: post-processing

### branching entropy는 좀 구해볼까? 
    
    L, right-side 
    R, left-side
    
    ## Complete
    
    >>> for w in ['대학', '대학생', '대학생과']:
    >>>     print(w, '%.3f' % lentropy.get(w, 0))

    대학 2.832
    대학생 1.721
    대학생과 0.000

In [594]:
min_noun_score = 0.5

In [595]:
score_dict = dict(scores)
s_ = score_dict['떡볶']
s = score_dict['떡볶이']
pretty(s_)
pretty(s)


Score(score=1.237, frequency=77, branching_entropy=0.366, feature_fraction=0.901, eojeol_fraction=0.000)
Score(score=3.160, frequency=67, branching_entropy=2.335, feature_fraction=0.925, eojeol_fraction=0.388)


### Nsub + J: 떡볶 + 이

    f(떡볶) ~= f(떡볶이): drop-rate, branching entropy로 하자
    (떡볶이 in Noun) and (이 in Josa)
    주로 한글자 짜리 조사가 문제 

In [596]:
josa = sorted(filter(lambda x:x[1] > -0.1, coefficient.items()), key=lambda x:x[1], reverse=True)
josa_dict = dict(josa)
len(josa_dict)

4295

In [597]:
' '.join(tuple(zip(*(filter(lambda x:x[1] > 0.1, josa))))[0])

'의 를 가 한 에 로 이 을 으로 적인 하고 된 에서 적 하는 할 적으로 이다 들은 에게 인 와 들이 였다 성 화 하게 까지 과 되는 에는 라고 처럼 해 부터 되어 하여 계 형 했다 로운 이나 이었다 만 들의 네 되고 생 나 론 이지 성을 해서 성이 치 스럽게 권 하지 제 해야 들 씩 질을 용 이야 될 없이 도 롭게 째 된다 하다 롭지 네는 예요 라는 한다 하기 층 되었다 들을 론적 하였다 화를 이라고 생이 론은 에서는 형의 마다 층의 화가 제를 권을 댁은 같이 든지 보다 론의 하면 계의 네가 되지 권의 시키는 인지 율을 류 됐다 형으로 님 돼 시하고 이면 생의 이가 당 스런 니까 스러운 화된 하면서 들에게 제는 권이 정책 엔 제의 쯤 적이고 이고 님이 라도 한테 론에서는 화하는 율 이라도 시켜 하며 님은 서도 층이 제가 질로 치를 시키고 율이 데 님을 시한 들과 으로는 였고 이지만 짜리 률 론에서 권에 상에서 에게는 인가 생을 화하고 데요 만한 보다는 이는 권은 까지나 일 하세요 상에 들도 화에 란 님의 에서도 들아 이랑 적으로는 계에 용을 시킬 생들이 하고라도 층을 한다는 화의 화와 생은 성과 제와 하던 론을 냐 죠 률이 루 되어야 시킨 생들의 론이 롭다 화하여 이냐 생들은 로는 대로 어요 계를 댁이 류의 라면 롭고 화는 율은 화할 로서 론적인 했던 성의 생과 성은 하거나 상 에다 여서 성에 되면 만의 화되어 했다는 된다고 시켰다 하십니까 네를'

In [341]:
postnoun = set('가감계과권꾼네님대댁당들력로론류률상생성시식용율의인이일재적제중째쯤층치판풍형화')
def find_composed_josa(josa):
    offset = set('계권꾼네님대력론류상생성시식용율적째쯤층치풍형화')
    return {j for j in josa if ((len(j) > 1) and (j[0] in josa) and not (j[0] in offset) and (j[1:] in josa))}

composed_josa = find_composed_josa(tuple(zip(*josa))[0])
jsub_set = {j[0] for j in composed_josa}

명사 뒤에 결합하여 의미를 확장하는 의존명사 (postnoun)이면서 조사에서는 잘 이용되지 않는 1음절 단어들을 offset으로 등록하였다. 

의존명사는 명사를 추출하는 질 좋은 R set feature이다. '나무 + 꾼' 처럼 '나무'가 명사라는 걸 잘 알려주기 때문이다. 하지만 실제로는 '나무꾼'을 단일 명사로 추출해야 한다. 

그 외에는 [-가, -이, -로, ...]의 경우 조사일 수도 있고, 다른 조사의 left-side 일수도 있다. 해당하는 조사들과 각각의 coefficient를 나열하면 아래와 같다. 

    >>> sorted({(j[0], '['+', '.join(tuple(zip(*sorted({'-'+j_[1:]:coef for j_,coef in coefficient.items() if len(j_) > 1 and j[0] == j_[0]}.items(), key=lambda x:x[1], reverse=True)))[0][:5])+']', '%.3f'%coefficient[j[0]]) for j in composed_josa}, key=lambda x:x[2], reverse=True)

    [('가', '[-에, -가, -의, -에서, -로]', '4.399'),
     ('한', '[-다, -테, -다는, -다고, -다면]', '3.766'),
     ('에', '[-서, -게, -는, -서는, -게는]', '3.551'),
     ('로', '[-운, -는, -서, -서는, -군]', '3.540'),
     ('이', '[-다, -나, -었다, -지, -야]', '3.281'),
     ('할', '[-만한, -지, -지도, -까봐, -수록]', '2.032'),
     ('인', '[-지, -가, -데, -들, -데요]', '1.411'),
     ('와', '[-의, -는, -도, -같이, -같은]', '1.398'),
     ('과', '[-는, -도, -의]', '1.137'),
     ('해', '[-서, -야, -진, -도, -서는]', '1.047'),
     ('만', '[-한, -의, -큼, -을, -은]', '0.791'),
     ('나', '[-마, -요, -면, -의, -구]', '0.723'),
     ('제', '[-를, -는, -의, -가, -와]', '0.630'),
     ('들', '[-은, -이, -의, -을, -에게]', '0.592'),
     ('씩', '[-이나, -은, -을, -만, -이라도]', '0.589'),
     ('도', '[-로, -가도, -다, -록]', '0.530'),
     ('당', '[-하고, -했다, -할, -하게, -하거나]', '0.257'),
     ('엔', '[-가, -지]', '0.240'),
     ('률', '[-이, -을, -은, -도, -과]', '0.171'),
     ('일', '[-까, -지라도, -지, -까요, -수록]', '0.154'),
     ('냐', '[-며, -는, -면은, -면, -라고]', '0.132'),
     ('랑', '[-은, -도]', '0.095'),
     ('케', '[-하는]', '0.065'),
     ('뿐', '[-인, -만, -이었다, -이다, -인데]', '0.031'),
     ('별', '[-로, -로는]', '0.025'),
     ('댁', '[-은, -이]', '0.018'),
     ('함', '[-으로써, -을, -이, -에, -과]', '0.018'),
     ('키', '[-로]', '0.012'),
     ('께', '[-서, -서는, -서도, -선, -로]', '0.007'),
     ('경', '[-에는, -에, -제, -제적]', '0.005'),
     ('됨', '[-에, -으로써, -을, -이, -과]', '0.004'),
     ('감', '[-이, -을]', '0.003'),
     ('덜', '[-이]', '0.001'),
     ('미', '[-국, -일]', '0.001'),
     ('임', '[-을, -에도, -이, -은, -에]', '0.000'),
     ('하', '[-고, -는, -게, -여, -지]', '0.000')]
     
그러므로 composed_josa의 첫글자들이 명사 후보의 맨 뒤에 붙어있으면 의심을 해보아야 한다

    >>> jsub_set = {j[0] for j in composed_josa}

    '가 감 경 과 께 나 냐 당 댁 덜 도 됨 들 랑 로 률 만 미 별 뿐 씩 에 엔 와 이 인 일 임 제 케 키 하 한 할 함 해'


In [598]:
def is_NsubJ(s_, s, bes_=0.5, f_fraction=0.5, eojeol_fraction_s_=0.1):
    return (s_.branching_entropy < bes_) and (s.frequency / s_.frequency > f_fraction) and (s_.eojeol_fraction < eojeol_fraction_s_)

pretty(score_dict['떡볶'])
pretty(score_dict['떡볶이'])

Score(score=1.237, frequency=77, branching_entropy=0.366, feature_fraction=0.901, eojeol_fraction=0.000)
Score(score=3.160, frequency=67, branching_entropy=2.335, feature_fraction=0.925, eojeol_fraction=0.388)


### [N + Jsub] + J: 대학생과 + 의

    f(대학생) >> f(대학생과)
    Right side branching entropy(대학생) ~= high
    (과의 in Josa) and (대학생 in Noun)

In [599]:
pretty(score_dict['대학'])
pretty(score_dict['대학생'])
pretty(score_dict['대학생과'])

Score(score=2.308, frequency=8678, branching_entropy=2.870, feature_fraction=0.785, eojeol_fraction=0.232)
Score(score=1.440, frequency=852, branching_entropy=1.988, feature_fraction=0.934, eojeol_fraction=0.242)
Score(score=6.211, frequency=26, branching_entropy=0.693, feature_fraction=1.000, eojeol_fraction=0.962)


In [650]:
is_jsubr_josa('상태로서', '의')

True

In [653]:
def is_NJsub(s_, s, bes_=0.5, bes=0.5, eojeol_fraction_s=0.5, f_fraction=0.3):
    return (s_.branching_entropy > bes_) and (s.branching_entropy < bes or s.eojeol_fraction > eojeol_fraction_s)

def fraction_of_jsubr_is_josa(l, jsub):
    rdict = {r:f for r, f in lrgraph.get(l+jsub, {}).items() if r != ''}
    f_sum = sum(rdict.values())
    if not f_sum:
        return 1.0 if jsub in josa_dict else 0.0
    f_jsubj = sum([f for r,f in rdict.items() if (jsub+r) in josa_dict])
    return (f_jsubj / f_sum)

def is_jsubr_josa(l, jsub, min_fraction=0.6):
    return fraction_of_jsubr_is_josa(l, jsub) > min_fraction

for w_, w in [('과학활동', '과학활동에'), ('지역주민', '지역주민과'), ('예술', '예술가'), ('상태', '상태로서')]:
    print(w_, w)
    pretty(score_dict[w_])
    pretty(score_dict[w])
    print('Nsub + J ? ', is_NsubJ(score_dict[w_], score_dict[w]))
    print('Jsub+R = J?', is_jsubr_josa(w_, w[len(w_):]), '%.3f'%fraction_of_jsubr_is_josa(w_, w[len(w_):]))
    print('N + Jsub ? ', is_NJsub(score_dict[w_], score_dict[w]), end='\n\n')

과학활동 과학활동에
Score(score=3.306, frequency=34, branching_entropy=1.655, feature_fraction=1.000, eojeol_fraction=0.000)
Score(score=6.211, frequency=12, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.833)
Nsub + J ?  False
Jsub+R = J? True 1.000
N + Jsub ?  True

지역주민 지역주민과
Score(score=1.899, frequency=75, branching_entropy=1.662, feature_fraction=0.970, eojeol_fraction=0.107)
Score(score=6.211, frequency=10, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.900)
Nsub + J ?  False
Jsub+R = J? True 1.000
N + Jsub ?  True

예술 예술가
Score(score=2.217, frequency=4386, branching_entropy=2.925, feature_fraction=0.822, eojeol_fraction=0.124)
Score(score=2.428, frequency=603, branching_entropy=2.549, feature_fraction=0.968, eojeol_fraction=0.124)
Nsub + J ?  False
Jsub+R = J? False 0.466
N + Jsub ?  False

상태 상태로서
Score(score=2.333, frequency=4624, branching_entropy=2.556, feature_fraction=0.997, eojeol_fraction=0.097)
Score(score=6.211, frequency=20, branching

In [601]:
candidates = dict(filter(lambda x:x[1].score > min_noun_score, score_dict.items()))
len(candidates)

57215

In [613]:
nouns = {}

for w, s in sorted(candidates.items(), key=lambda x:len(x[0])):
    n = len(w)
    if n == 1:
        nouns[w] = s
        continue
    # Case: 떡볶 + 이 --> remove 떡볶
    if (w[-1] in postnoun) and (w[:-1] in nouns):
        s_ = nouns[w[:-1]]
        if is_NsubJ(s_, s):
            del nouns[w[:-1]]
    
    # Case: [대학생 + 과] + 의 --> pass 대학생과 e = 1
    # Case: [정국 + 으로] + 의 --> pass 정국으로 e = 2
    NJsub = False
    for e in range(1, max(5, n)):
        l = w[:-e]
        jsub = w[-e:]
        if (l in nouns) and (jsub in josa_dict):
            s_ = nouns[l]
            if is_NJsub(s_, s):
                NJsub = True
                break
            if 0 == sum((1 if (jsub+r) in josa_dict else 0 for r in lrgraph.get(l, {}).keys())):
                NJsub = True
                break
    if NJsub:
        continue

    nouns[w] = s

len(candidates), len(nouns)

(57215, 48207)

In [669]:
words = ['예술가', '과거', '정국', '취재', '개체', '배후']
for w in words:
    print(w, w in nouns)

예술가 True
과거 True
정국 True
취재 True
개체 True
배후 True


In [673]:
'  '.join(tuple(zip(*sorted(nouns.items(), key=lambda x:x[1].score, reverse=True)[:100]))[0])

'함북  김성환  학제간  제야  김태정  덮어놓고  지오그래픽  본모  앨리  각권  자와  로버트  언어상  저스틴  권영길  각급  금융상  인조견  전대미문  런닝머신  피천득  푸나  란상  한낱  벽산  환경주  내부자  무호흡  현도  함경남도  주간한국  천혜  까자  국립과  혈중  노트르담  문화교호간  쌍마자동차  불굴  민흘림  아방  멸망시  제반  유종호  외환수급  석박사  클린업  가고파  샤를  모스  재판과정에서  지역환경  김덕수  현종  혼외  영과  우르  열흘간  해마다  소수인  만반  세계여성  이인국  아사히  문학비  레포  김근태  신체상  순위항목  아더  탐관  이혜경  가향  마에다  산업간  화랭이패  나노  김영동  인터컨티넨탈  다민족  최현배  국가교호간  도큐  수간  천의  페레  생산자물가  탈자본주  우국  피파  신윤복  오륙십  하루종일  미체험  완전식  여러번  파죽  생면  루츠  맨하탄'

### Compound: 소수 + [집단 + 의]

    소수 + 집단의 (집단의 = 집단 + 의 인지 확인)
    Noun score 대체


In [664]:
X[:,1359].nonzero()[0]

array([   32,    76,   101,   248,   316,   468,   654,   735,  1175,
        1246,  1248,  1254,  1447,  1571,  1630,  1750,  1766,  1786,
        1790,  1808,  1929,  1941,  2021,  2233,  2263,  2268,  2295,
        2827,  2841,  2873,  2945,  3126,  3318,  3496,  3550,  3630,
        3712,  3741,  3762,  3809,  3877,  3886,  3900,  3972,  4053,
        4054,  4171,  4452,  4493,  4530,  4585,  4699,  4703,  4740,
        4800,  4818,  4871,  4872,  4896,  4979,  5139,  5210,  5237,
        5316,  5365,  5431,  5450,  5484,  5493,  5529,  5542,  5853,
        5943,  5953,  6234,  6251,  6292,  6329,  6400,  6430,  6545,
        6587,  6654,  6671,  6718,  6732,  6757,  6784,  6898,  6954,
        6974,  7153,  7192,  7193,  7206,  7273,  7300,  7343,  7356,
        7432,  7498,  7560,  7561,  7615,  7663,  7665,  7773,  7834,
        8050,  8079,  8081,  8182,  8184,  8226,  8425,  8448,  8477,
        8537,  8636,  8756,  8777,  8855,  8897,  9030,  9120,  9204,
        9234,  9258,

In [678]:
list(zip(x_words, y))

[('겨드랑이', 1),
 ('끓', -1),
 ('간첩', 1),
 ('무서', -1),
 ('쇠퇴', 1),
 ('호흡기', 1),
 ('구겨', -1),
 ('천문학', 1),
 ('두들겨', -1),
 ('추위', 1),
 ('참가자', 1),
 ('단기기억', 1),
 ('국사', 1),
 ('출품', 1),
 ('위엥찬', 1),
 ('주치의', 1),
 ('짚', -1),
 ('파뷔에', 1),
 ('계통', 1),
 ('전생', 1),
 ('관점', 1),
 ('절망', 1),
 ('전세', 1),
 ('두들기', -1),
 ('시국', 1),
 ('소모', 1),
 ('노정', 1),
 ('이어진', -1),
 ('울분', 1),
 ('색칠', 1),
 ('날', 1),
 ('씌우', -1),
 ('영향', 1),
 ('확보', 1),
 ('비해', -1),
 ('파탄', 1),
 ('곤욕', 1),
 ('꿀', 1),
 ('귀의', 1),
 ('추진력', 1),
 ('남자애', 1),
 ('파란색', 1),
 ('일념', 1),
 ('일체', 1),
 ('편집장', 1),
 ('전동차', 1),
 ('일부분', 1),
 ('얼', -1),
 ('파라오', 1),
 ('웃겼', -1),
 ('꾸리찌바', 1),
 ('각국', 1),
 ('최고위원회의', 1),
 ('햄릿', 1),
 ('만개', 1),
 ('선우완', 1),
 ('뚜껑', 1),
 ('지구촌', 1),
 ('고전', 1),
 ('찢겨', -1),
 ('창틀', 1),
 ('김구', 1),
 ('안기', -1),
 ('한쪽', 1),
 ('영월', 1),
 ('근성', 1),
 ('엽기', 1),
 ('뒤엎', -1),
 ('친해', -1),
 ('변조', 1),
 ('보호', 1),
 ('저', -1),
 ('동북아시아', 1),
 ('큰집', 1),
 ('독일', 1),
 ('인근', 1),
 ('출신', 1),
 ('부둥켜안', -1),
 ('예식장', 1),
 ('국회의원

In [667]:
for word, freq in sorted(rlgraph['이었다고'].items(), key=lambda x:x[1], reverse=True)[:50]:
    if not word in x_word_to_int: continue
    print(word, y[x_word_to_int.get(word,-1)], freq)

것 1 48
때문 1 27
사람 1 10
일 1 9
사건 1 6
길 -1 5
원인 1 5
과정 1 4
편 -1 4
명 1 4
노선 1 4
곳 1 4
요인 1 4
책 1 4
연속 1 3
꿈 1 3
전 1 3
표현 1 3
분 1 3
산물 1 3
싸움 1 3
위함 -1 3
선택 1 3
방법 1 3
뿐 1 3
집 1 3
전쟁 1 2
음식 1 2
소산 1 2
경험 1 2
전환 1 2
모임 1 2
생각 1 2
지경 1 2
대상 1 2
죽음 1 2
아들 1 2
중 1 2
입장 1 2
그것 1 2
상황 1 2
흐름 1 2
운동 1 2
원 1 2
반응 1 2
소설 1 2
